In [96]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import sqlite3
from tqdm import tqdm

In [97]:
#function to get the speeches from the link
def get_speech(link):
    page = urlopen(link)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    text = soup.find(class_="field-docs-content").get_text()
    #removing all \n and \' from the text. will need to do more preprocessing to it later
    text = text.replace("\n", "").replace("\'", "")
    return(text)


In [98]:
#creating a sqlite database to store all of the speeches and some info about the presidents
#TODO change path to correct data path
con = sqlite3.connect("speeches.db")
cur = con.cursor()
cur.execute("CREATE TABLE presidents(name, term, lived, party)")
cur.execute("CREATE TABLE speeches(president, speech_date, speech)")
con.commit()


In [100]:
index_url = "https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/inaugural-addresses"
page = urlopen(index_url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [101]:
#getting all of the speeches and putting them into a list
to_insert = []
table = soup.find(class_="field-body")
for link in tqdm(table.find_all("tr")):
    name = link.find("strong")
    if(type(name) != type(None)):
        prez = name.get_text()
    url = link.find("a")
    if(type(url) != type(None)):
        date = url.get_text()
        href = url.get("href")
        speech = get_speech(href)
    try:
        to_insert.append([prez, date, speech])
    except:
        pass



100%|██████████| 65/65 [00:36<00:00,  1.80it/s]


In [102]:
#removing the first entry beccause it is an error
to_insert.remove(to_insert[0])
#populating the speech table in the database
for x in to_insert:
    cur.execute("INSERT INTO speeches VALUES('" + x[0] + "', '" + x[1] + "', '" + x[2] + "')")

con.commit()

In [103]:
#getting the info on the presidents
wiki_url = "https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States"
page = urlopen(wiki_url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [107]:
#adding all of the info on the presidents to a list
prez_list = []
table = soup.find("tbody")
for t in table.find_all("tr"):
    num = 0
    for x in t.find_all("td"):
        #print(x.get_text())
        if(num == 1):
            try:
                name = x.get_text().split("(")[0].replace("\n", "")
                life = x.get_text().split("(")[1].split(")")[0].replace("\n", "")
            except:
                pass
        if(num == 2):
            try:
                term = x.get_text().replace("\n", "")
            except:
                pass
        if(num == 4):
            try:
                party = x.get_text().replace("\n", "")
            except:
                pass
        num+=1
    prez_list.append([name, term, life, party])



SyntaxError: '(' was never closed (649311801.py, line 16)

In [105]:
#removing the first entry from the prez_list
prez_list.remove(prez_list[0])

#adding the presidents to the database
for x in prez_list:
    cur.execute("INSERT INTO presidents VALUES('" + x[0] + "', '" + x[1] + "', '" + x[2] + "', '" + x[3]+ "')")


con.commit()

In [95]:
#seeing if all info in the database is correct
#res = cur.execute("SELECT * from speeches")
#res.fetchall()

res = cur.execute("SELECT * from presidents")
res.fetchall()


[('George Washington',
  'April 30, 1789–March 4, 1797',
  '1732–1799',
  'Unaffiliated'),
 ('John Adams', 'March 4, 1797–March 4, 1801', '1735–1826', 'Federalist'),
 ('Thomas Jefferson',
  'March 4, 1801–March 4, 1809',
  '1743–1826',
  'Democratic-Republican'),
 ('James Madison',
  'March 4, 1809–March 4, 1817',
  '1751–1836',
  'Democratic-Republican'),
 ('James Monroe',
  'March 4, 1817–March 4, 1825',
  '1758–1831',
  'Democratic-Republican'),
 ('John Quincy Adams',
  'March 4, 1825–March 4, 1829',
  '1767–1848',
  'Democratic-Republican[f]National Republican'),
 ('Andrew Jackson', 'March 4, 1829–March 4, 1837', '1767–1845', 'Democratic'),
 ('Martin Van Buren',
  'March 4, 1837–March 4, 1841',
  '1782–1862',
  'Democratic'),
 ('William Henry Harrison',
  'March 4, 1841–April 4, 1841[e]',
  '1773–1841',
  'Whig'),
 ('John Tyler',
  'April 4, 1841[i]–March 4, 1845',
  '1790–1862',
  'Whig[j]Unaffiliated'),
 ('James K. Polk', 'March 4, 1845–March 4, 1849', '1795–1849', 'Democratic'),

In [106]:
con.close()

In [ ]:
#TODO need to make sure that the presidents name in both the speeches table and the presidents table are the same